In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [3]:
from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

In [ ]:
Loading csv files

In [ ]:
users = pd.read_csv('')
pois = pd.read_csv('C:\Users\lasse\Desktop\RecommenderDL\TIST2015_POIs.csv')

In [ ]:
#dataset_TIST2015_Checkins: User ID, Venue ID, UTC time, Time zone
#dataset_TIST2015_POIs: Venue ID, Latitude, Longitude, Venue category name, Country code 
#dataset_TIST2015_Cities: Cityname, Latitude, Longitude, Vanue Category, Country code, Country name, City type
#dataset_UbiComp2016_UserProfile_{CITY}: User ID, Gender, Twitter friend count, Twitter follower count

#Scenario 1: Checkins og POI locations

checkin_cols = ['poi_id', 'timestamp', 'latitude', 'longitude']
venue_cols = ['venue_id', 'latitude', 'longitude', 'category' 'unix_timestamp']

#Scenario 2: Users og Checkins

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'gender', 'sex']
ratings_cols = ['user_id', 'venue_id', 'latitude', 'longitude', 'category' 'unix_timestamp']

#Scenario3: Users, Checkins og POI Locations
checkin_cols = ['poi_id', 'gender', 'sex']
venue_cols = ['venue_id', 'user_id', 'latitude', 'longitude', 'category' 'unix_timestamp']
user_cols = ['user_id', 'gender']


